In [1]:
import utils.github_util as gu
from github import Github
import pandas as pd

In [ ]:
# g = Github("")


In [ ]:
# ./files/memleak_files/index.csv
#  _id + url + tag

from utils import mongo_helper

mongo = mongo_helper.MongoHelper()
prs_df = pd.DataFrame(mongo.db["data"].find())
simplified_prs_df = prs_df[["url", "tag"]]
simplified_prs_df.to_csv("./../files/memleak_files/index.csv")

In [ ]:

import os

try:
    os.chdir("./../files/memleak_files")
    prs = pd.read_csv("./index.csv")
    
    count = len(prs.index)
    print(count)

    target = prs.iloc[0:1100]
    
    for index, row in target.iterrows():
        url = row['url']
        print(f"Downloading files of index {index} : {url}")
        owner, repo, pull_number = gu.get_info_from_github_url(url)
        dir_name = f"{owner}-{repo}-{pull_number}"
        os.makedirs(dir_name, mode=0o777, exist_ok=True)

        patch_diff_url = f"https://patch-diff.githubusercontent.com/raw/{owner}/{repo}/pull/{pull_number}.diff"
        with open(f"./{dir_name}/patch.diff", 'wb') as diff_file:
            diff_content = gu.get_raw_file_content(patch_diff_url)
            diff_file.write(diff_content)
            diff_file.close()

        commits_of_pr = []
        commits_info_list = gu.get_pr_commits(owner, repo, pull_number)
        for commit_info in commits_info_list:
            commits_of_pr.append(commit_info["sha"])
        last_commit_of_pr = commits_of_pr[-1]
        
        file_counter = 0
        

        for file_info in gu.get_files_changed_in_pr(owner, repo, pull_number):
            file_path = file_info["filename"]
            file_name = os.path.basename(file_path)

            if file_name.endswith((".c", ".cpp", ".h", ".hpp", ".cc")):
                file_counter += 1
                change_history = gu.get_file_changes_with_branch(owner, repo, file_path, last_commit_of_pr)
                oldest_sha_index = -1
                for change in change_history:
                    if change["sha"] in commits_of_pr:
                        oldest_sha_index = change_history.index(change)

                if oldest_sha_index == -1:
                    print(f"Cannot find history version of file {file_name}!")
                    continue
                if oldest_sha_index < (len(change_history)-1):
                    oldest_sha_index += 1
                target_sha = change_history[oldest_sha_index]["sha"]
        
                saved_file_name = f"{file_name}"
                with open(f"./{dir_name}/{saved_file_name}", 'wb') as f:
                    bug_file_content = gu.get_file_content_in_specific_commit(owner, repo, file_path, target_sha)
                    f.write(bug_file_content)
                    f.close()
                
            else:
                continue
                
        print(f"Success: saved {file_counter} files.")
finally:
    
    os.chdir("./../../downloader")